In [16]:
# 경고(warning) 비표시
import warnings
warnings.filterwarnings('ignore')

# 고객의 전체모습 파악

### 1 : 데이터 읽고 확인

In [17]:
import pandas as pd

use_log = pd.read_csv('./data/use_log.csv')
customer_master = pd.read_csv('./data/customer_master.csv')
class_master = pd.read_csv('./data/class_master.csv')
campaign_master = pd.read_csv('./data/campaign_master.csv')


In [18]:
use_log.head() # 이용 이력



,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [19]:
customer_master.head() # 회원 데이터 , is_deleted -> 탈퇴한 회원의 정보 파악

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


In [20]:
class_master.head() # 회원 구분 데이터 (종일,주간,야간)

,class,class_name,price
0,C01,0_종일,10500
1,C02,1_주간,7500
2,C03,2_야간,6000


In [21]:
campaign_master.head() # 행사 구분 데이터(입회비 유무 등)

,campaign_id,campaign_name
0,CA1,2_일반
1,CA2,0_입회비반액할인
2,CA3,1_입회비무료


### 2 : 고객 데이터를 가공

In [22]:
# customer_master +  class.master

customer_join = pd.merge(customer_master,class_master,on='class',how='left')



In [23]:
# customer_join + campaign_master
customer_join = pd.merge(customer_join,campaign_master,on='campaign_id',how='left')

In [24]:
customer_join.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,2_야간,6000,2_일반


In [25]:
print(len(customer_join))
print(len(customer_master))

4192
4192


In [26]:
# end_data : 회원의 탈퇴만 nan값있고 나머지는 없음

customer_join.isnull().sum()

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

### 3 : 고객데이터를 집계

* 어떤 회원?
* 어떤 캠페인이 많은지
* 남녀 비율은 얼마나 되는지?
* 현재 가입된 회원과 탈퇴한 회원
* 언제 입회/탈퇴가 많았는지?
* 탈퇴할 때까지의 기간은 어느정도인지? :is_delete열

In [34]:
customer_join.groupby('class_name').count()['customer_id'] # 어떤 회원

class_name
0_종일    2045
1_주간    1019
2_야간    1128
Name: customer_id, dtype: int64

In [33]:
customer_join.groupby('campaign_name').count()['customer_id'] # 어떤 캠페인

campaign_name
0_입회비반액할인     650
1_입회비무료       492
2_일반         3050
Name: customer_id, dtype: int64

In [41]:
customer_join.groupby('gender').count()['customer_id'] # 남녀 비율

gender
F    1983
M    2209
Name: customer_id, dtype: int64

In [40]:
customer_join.groupby('is_deleted').count()['customer_id'] # 탈퇴한 회원 
                                                           # 0: 가입, 1: 탈퇴

is_deleted
0    2842
1    1350
Name: customer_id, dtype: int64

In [43]:
# 20180401 이후 가입한 회원
customer_join['start_date'] = pd.to_datetime(customer_join['start_date'])
customer_start = customer_join.loc[customer_join['start_date'] > pd.to_datetime('20180401')]
print(len(customer_start)) # 1361명이 20180401 이후 가입한 회원

1361


In [44]:
len(customer_join)

4192

In [36]:
customer_counts = customer_join.groupby('start_date').count()['customer_id'] # 입회 내림차순

customer_counts_sorted = customer_counts.sort_values(ascending=False)

customer_counts_sorted

start_date
2017-12-01 00:00:00    160
2017-08-01 00:00:00    122
2018-01-01 00:00:00    122
2017-09-01 00:00:00    115
2017-11-01 00:00:00    108
                      ... 
2018-10-02 00:00:00      1
2018-10-04 00:00:00      1
2019-03-10 00:00:00      1
2019-02-02 00:00:00      1
2019-03-15 00:00:00      1
Name: customer_id, Length: 215, dtype: int64

In [37]:
customer_counts = customer_join.groupby('end_date').count()['customer_id'] # 탈퇴 내림차순

customer_counts_sorted = customer_counts.sort_values(ascending=False)

customer_counts_sorted

end_date
2019-02-28 00:00:00    149
2019-01-31 00:00:00    129
2018-05-31 00:00:00    125
2018-04-30 00:00:00    121
2018-08-31 00:00:00    113
2018-12-31 00:00:00    111
2019-03-31 00:00:00    111
2018-06-30 00:00:00    106
2018-09-30 00:00:00    104
2018-07-31 00:00:00    102
2018-10-31 00:00:00     90
2018-11-30 00:00:00     89
Name: customer_id, dtype: int64

### 4 : 최근 고객데이터를 집계

In [50]:
# 2019년 3월의 고객 데이터 파악 /// 탈퇴 시 월말까지 신청하면 다음 달 말에 탈퇴가 됨

customer_join['end_date'] = pd.to_datetime(customer_join['end_date'])

customer_newer = customer_join.loc[(customer_join['end_date']>=pd.to_datetime("20190331")) |
                                   (customer_join['end_date'].isna())]

print(len(customer_newer))
customer_newer['end_date'].unique()

2953


<DatetimeArray>
['NaT', '2019-03-31 00:00:00']
Length: 2, dtype: datetime64[ns]

In [51]:
# 2019년 3월 고객
customer_newer.groupby('class_name').count()['customer_id']

class_name
0_종일    1444
1_주간     696
2_야간     813
Name: customer_id, dtype: int64

In [52]:
customer_newer.groupby('campaign_name').count()['customer_id']

campaign_name
0_입회비반액할인     311
1_입회비무료       242
2_일반         2400
Name: customer_id, dtype: int64

In [53]:
customer_newer.groupby('gender').count()['customer_id']

gender
F    1400
M    1553
Name: customer_id, dtype: int64

### 5 : 이용이력 데이터를 집계

* 한달 이용 횟수와 변화
* 회원이 스포츠 센터를 정기적으로 이용하는지에 대한 여부

In [ ]:
# 월 이용 횟수의 평균값 , 중앙값, 최댓값, 최소값
# 정기적 이용 여부를 플래그로 작성

In [60]:
# 고객마다 연월 이용 횟수를 집계


# usedate를 datetime으로 바꾸기
use_log['usedate'] = pd.to_datetime(use_log['usedate'])
use_log['연월'] = use_log['usedate'].dt.strftime('%Y%m')

# 연월과 id로 그룹화
use_log_months = use_log.groupby(['연월','customer_id'], as_index=False).count()

use_log_months.rename(columns={"log_id":"count"}, inplace=True)

del use_log_months["usedate"]
use_log_months.head()


,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [61]:
use_customer = use_log_months.groupby('customer_id').agg(["mean","median",'max','min'])['count']
use_customer = use_customer.reset_index(drop=False)
use_customer.head()

,customer_id,mean,median,max,min
0,AS002855,4.500000,5.0,7,2
1,AS008805,4.000000,4.0,8,1
2,AS009013,2.000000,2.0,2,2
3,AS009373,5.083333,5.0,7,3
4,AS015233,7.545455,7.0,11,4


### 6 : 이용이력 데이터로부터 정기이용 여부  플래그를 작성

In [64]:
# 고객마다 월/요일별 집계

use_log['weekday'] = use_log['usedate'].dt.weekday
# use_log['weekday']

use_log_weekday = use_log.groupby(['customer_id','연월','weekday'],
                                  as_index=False).count()[['customer_id','연월','weekday','log_id']]

use_log_weekday.rename(columns={'log_id':'count'},inplace=True)
use_log_weekday.head()

,customer_id,연월,weekday,count
0,AS002855,201804,5,4
1,AS002855,201805,2,1
2,AS002855,201805,5,4
3,AS002855,201806,5,5
4,AS002855,201807,1,1


In [66]:
# 월별 정기적 이용 여부, 고객마다 월/요일별 집계 최대값이 4이상인 요일이 있으면 플래그를 1로 처리

use_log_weekday = use_log_weekday.groupby('customer_id', as_index=False).max()[['customer_id','count']]


use_log_weekday['routine_flg'] = 0

use_log_weekday['routine_flg'] = use_log_weekday['routine_flg'].where(use_log_weekday['count']<4 , 1)

use_log_weekday.head() 

,customer_id,count,routine_flg
0,AS002855,5,1
1,AS008805,4,1
2,AS009013,2,0
3,AS009373,5,1
4,AS015233,5,1


### 7 : 고객 데이터와 이용이력데이터를 결합

### 8 : 회원기간 계산

### 9 : 고객행동의 각종통계량을 파악

### 10 : 탈퇴회원과 지속회원의 차이를 파악